In [13]:
import os
os.environ['OPENAI_API_KEY']="API_KEY" # À Modifier

In [14]:
from langchain_openai import OpenAI

# initialize the models
llm = OpenAI(
    model_name="gpt-3.5-turbo-instruct",
)

In [15]:
example = [
{"query" : "analyse de l'évolution du de PNB trimestriel entre T2-22 et T2-23 ? ",
 "answer" : "Explication"
},{
"query" : "rationnel de l'évolution du de PNB trimetriel entre T2-22 et T2-23 ?",
"answer": "Explication"
},{
"query" : "Singularité de la trajectoire de PNB entre T2-22 et T2-23 ?",
"answer" : "Description"
},{
"query" : "analyse comparée des trajectoires de PNB et de FG entre T2-22 et T2-23 ? ",
"answer" : "Comparaison"
}
]

In [16]:
from langchain import FewShotPromptTemplate
from langchain import PromptTemplate

example_template = """
Question: {query}
Réponse: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template)


prefix="""Classe la question posée simplement dans l'une des trois catégories suivantes : Explication, Description, ou Comparaison. Si tu ne sais pas, réponds  'jsp'"""

suffix= """
User : {query}
AI:"""
     
few_shot_prompt_template = FewShotPromptTemplate(
    examples=example,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n")
    
    

In [17]:
query="Peux-tu identifier les singularités de la trajectoire mensuelle de FG en 2023"


In [18]:
réponse = llm.invoke(few_shot_prompt_template.format(query=query))

In [19]:
print(réponse)

 jsp


In [23]:
example2 = [
{"query" : "analyse de l'évolution du PNB trimestriel entre T2-22 et T2-23 ? ",
 "answer" : "Objet1 : PNB, Objet2 : Non Précisé, période départ : T2-22 , Période fin : T2-23 , Fréquence : trimestrielle"
},{
"query" : "rationnel de l'évolution des FG mensuels entre T2-22 et T2-23 ?",
 "answer" : "Objet1 : FG , Objet : Non précisé ,période départ : T2-22 , Période fin : T2-2 , Fréquence : mensuelle"
},{
"query" : "Singularité de la trajectoire semi-annuelle de frais de personnel entre H1-20 et H2-24 ?",
 "answer" : "Objet1 : Frais de personnel , Objet2 : Non précisé , période départ : T2-22 , Période fin : T2-23 , Fréquence : semi-annuelle"
},{
"query" : "analyse comparée des trajectoires des dépenses et du PNB entre T2-22 et T2-23 ? ",
 "answer" : "Objet1 : Dépenses , Objet2 : PNB , période départ : T2-22 , Période fin : T2-2 , Fréquence : Non précisée"
}
]

In [24]:
example_template = """
Question: {query}
Réponse: {answer}
"""

example_prompt2 = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template)


prefix2="""Peux-tu extraire les 'paramètres' de cette {réponse} - Réponds de la façon la plus succincte possible"""

suffix2= """
User : {query}
AI:"""
     
few_shot_prompt_template2 = FewShotPromptTemplate(
    examples=example2,
    example_prompt=example_prompt2,
    prefix=prefix2,
    suffix=suffix2,
    input_variables=["réponse","query"],
    example_separator="\n\n")
    

In [25]:
réponse2 = llm.invoke(few_shot_prompt_template2.format(réponse=réponse,query=query))

In [27]:
print(réponse2)

 Objet1 : FG, Objet2 : Non précisé, période départ : Janvier 2023, période fin : Décembre 2023, Fréquence : mensuelle


In [30]:
import json

# Séparation des informations en utilisant la virgule comme délimiteur
infos = [info.strip() for info in réponse2.split(',')]

# Création d'un dictionnaire pour stocker les informations
dictionnaire_infos = {}
for info in infos:
    cle, valeur = info.split(':')
    dictionnaire_infos[cle.strip()] = valeur.strip()

# Conversion du dictionnaire en format JSON
json_resultat = json.dumps(dictionnaire_infos, ensure_ascii=False)

# Affichage du résultat au format JSON
print(json_resultat)

{"Objet1": "FG", "Objet2": "Non précisé", "période départ": "Janvier 2023", "période fin": "Décembre 2023", "Fréquence": "mensuelle"}


In [37]:
dictionnaire_infos = json.loads(json_resultat)

# Accéder à un élément du dictionnaire
print(dictionnaire_infos["période départ"])  # Affiche : FG


Janvier 2023
